In [ ]:
import project_lib as mylib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import scipy as sp
%matplotlib notebook
#mylib.rl()

In [ ]:
from matplotlib import cm
def f_blues(points):
    spec = cm.nipy_spectral#cm.Blues
    return [ spec(x) for x in np.linspace(0, 1, points) ]

def f_reds(points):
    spec = cm.nipy_spectral#cm.Reds
    return [ spec(x) for x in np.linspace(0, 1, points) ]

def normalised_vesuvius_schedule(points):
    AB = mylib.vesuvius_schedule(points)
    AB[-1] =AB[-1]/max(AB[-1])
    AB[0]= AB[0]/max(AB[0])
    return AB

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# PREDEFINED CONSTANTS
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
points = 1000
qubits = 4
h_i = [(-1)**i * i for i in range(1,1+qubits)]
J_ij = np.triu(np.asarray([[(-1)**j * i**2 + (-1)**i * j**2 for i in range(1,1+qubits)] for j in range(1,1+qubits)]), k=1)
ss = np.linspace(0,1,points)
kss = ss[1:-1]
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# SCHEDULES
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
schedules = {
            'const. kappa': 'constant_k',
            'linear': mylib.linear_schedule,
            'fast exp': mylib.exponential_schedule_ab(50,50),
            'slow exp': mylib.exponential_schedule_ab(2,2),
            'vesuvius': normalised_vesuvius_schedule,
            'inv. energy': 'inverse_energy',
            'inv. sqr. energy': 'inverse_square_energy',
            }

sched_by_num = {
    0: 'const. kappa',
    1: 'linear',
    2: 'inv. sqr. energy',
    3: 'inv. energy',
    4: 'vesuvius',
    5: 'slow exp',
    6: 'fast exp',   
}


#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# COLOURS TO USE
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
blues = f_blues(len(schedules)+2)
reds = f_reds(len(schedules)+2)
    


In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ANNEALING
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
T = 2.**(np.arange(15)-2)/10


psn = {}
for n in sched_by_num:
    print 'Runing schdule : (%s out of %s)'%(n+1,len(sched_by_num))
    ps_sched = []
    for i,t in enumerate(T):
        print 'Time Run : (%s/%s)'%(i+1,len(T))
        a = mylib.Anneal(qubits,[h_i,J_ij],T=t,points = points,sched_func = schedules[sched_by_num[n]],light = True)
        a.run()
        ps_sched.append(a.problem_x0_prob[-1])
    psn[n] = np.asarray(ps_sched)
    

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# COLOURS TO USE
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from matplotlib import cm
def f_blues(points):
    spec = cm.nipy_spectral#cm.Blues
    return [ spec(x) for x in np.linspace(0, 1, points) ]

blues = f_blues(len(schedules)+1)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# PLOTTING THE SCHEDULES SUCCESS RATE WITH TIME
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

plt.figure(figsize = (13,6))
axA=plt.subplot2grid((1,4),(0,0),colspan = 3)
for n in sched_by_num:
    psn[n]
    axA.semilogx(T,psn[n], label = str(n)+' : '+sched_by_num[n], color = blues[n])
    
axA.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.,title='$Schedule$')
axA.set_ylabel(r'$P($success$)$')
axA.set_xlabel(r'$ \tau $')
plt.show()